<a href="https://colab.research.google.com/github/FernandoElizarraras7/SIMULACION-II/blob/main/VARIABLES_ANTITETICAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REDUCCION DE VARIANZA POR VARIABLES ANTITETICAS
La idea básica es generar muestras en pares correlacionados negativamente para reducir la varianza del estimador.

Para la integral con $x \sim U(0,1)$ y función $f(x)$:
- Generamos $u_i \sim U(0,1)$ y su antitético $1-u_i$.
- Formamos la media por par:
$$Z_i = \tfrac{1}{2}\big[f(u_i) + f(1-u_i)\big]$$
- El estimador con antitéticas es:
$$\hat I_{\text{anti}} = \frac{1}{m} \sum_{i=1}^m Z_i$$
donde $N = 2m$ es el total de simulaciones ordinarias equivalentes en comparación.
- La varianza del estimador con antitéticas es:
$$\mathrm{Var} (\hat I_{\text{anti}}) = \frac{\mathrm{Var}(Z)}{m}$$
- Porcentaje de reducción de la varianza (relativo al Mc crudo):
$$\text{Reducción (%)} = 100 \cdot \frac{\mathrm{Var}(\hat I_{\text{crudo}}) - \mathrm{Var}(\hat I_{\text{anti}})}{\mathrm{Var}(\hat I_{\text{crudo}})}$$


In [7]:
# MC Crudo vs MC con Variables Antitéticas

import numpy as np
import time

# Se fija semilla para reproducibilidad
np.random.seed(1000)

# Se define la función a integrar
def f(x):
  return np.sqrt(np.arctan(x))

# Número total de muestras (se asegura que sea par)
N = 10**5
if N % 2 != 0:
  N += 1

# === Monte Carlo Crudo ===
t1_start = time.time()

x = np.random.rand(N)  # Muestras en [0,1]
fx = f(x)  # Se evalúa en f(x)
I_crudo = np.mean(fx)  # Promedio
Var_crudo = np.var(fx, ddof=1) / N  # Varianza del estimador
Std_crudo = np.sqrt(Var_crudo)  # Desviación estándar

t1_end = time.time()
t1 = t1_end - t1_start

print("=== Método Crudo ===")
print(f"Estimación Integral = {I_crudo:.6f}")
print(f"Varianza = {Var_crudo:.6e}")
print()

# === Monte Carlo con Variables Antitéticas ===
t2_star = time.time()

m = N // 2 # Número de pares
u = np.random.rand(m) # Mitad de muestras
u_bar = 1 - u # Antitéticas

# Promedio por par
z_pairs = 0.5 * (f(u) + f(u_bar))

I_anti = np.mean(z_pairs) # Estimador antitético
Var_anti = np.var(z_pairs, ddof=1) / m # Varianza del estimador

t2_end = time.time()
t2 = t2_end - t2_star

print("=== Método con Variables Antitéticas ===")
print(f"Estimación Integral = {I_anti:.6f}")
print(f"Varianza = {Var_anti:.6e}")
print()

# === Comparación ===
reduction_pct = 100 * (Var_crudo - Var_anti) / Var_crudo
print("=== Comparación ===")
print(f"Reducción relativa de varianza = {reduction_pct:.2f} %")

=== Método Crudo ===
Estimación Integral = 0.630305
Varianza = 4.202387e-07

=== Método con Variables Antitéticas ===
Estimación Integral = 0.630084
Varianza = 5.547902e-08

=== Comparación ===
Reducción relativa de varianza = 86.80 %


## Observaciones sobre la reducción obtenida (86.8%)

- La técnica de variables antitéticas produjo una reducción relativa de la varianza del 86.8%, lo cual es una mejora significativa.
- Esto se explica porque la función $f(x)=\sqrt{\arctan(x)}$ es monótona creciente en $[0,1]$. Las parejas $u$ y $1-u$ generan valores fuertemente anticorrelados de $f$, y al promediar por par se atenúan las fluctuaciones.
- Se recomienda verificar la estabilidad del resultado repitiendo la simulación con distintas semillas (por ejemplo 50–200 repeticiones). Además es útil calcular la correlación muestral entre $f(u)$ y $f(1-u)$ como prueba de la causa de la reducción.
- La aplicación de antitéticas al método MC Acierto y Error es teóricamente factible pero su eficacia no está garantizada debido a la naturaleza discreta del estimador; conviene evaluarlo empíricamente.

En conclusión, la técnica de variables antitéticas es altamente recomendable para integrales de funciones monótonas como la trabajada, pues permite alcanzar la misma precisión con una fracción de las simulaciones necesarias.

In [12]:
# Monte Carlo Acierto y Error
t3_start = time.time()

#Máximo de la función en [0,1]
x_grid = np.linspace(0,1,10000)
f_max = np.max(f(x_grid))

# Generar puntos aleatorios
x_rand = np.random.rand(N)
y_rand = np.random.rand(N) * f_max

# Contar cuántos puntos caen debajo de la curva
hits = (y_rand <= f(x_rand)).astype(float)

I_hitmiss = np.mean(hits) * f_max # Estimación de la integral
Var_hitmiss = p*(1-p)*(f_max**2) / N # Varianza

t3_end = time.time()
t3 = t3_end - t3_start

print("=== Método Acierto y Error ===")
print(f"Estimación Integral = {I_hitmiss:.6f}")
print(f"Varianza = {Var_hitmiss:.6e}")
print()

# Monte Carlo Acierto y Error con Variables Antitéticas
t4_start = time.time()

# Se usan N pares, se divide entre 2
M = N // 2

# Se generan puntos uniformes en [0,1] y sus antitéticos
u = np.random.rand(M)
u_bar = 1 - u
v = np.random.rand(M)
v_bar = 1 - v

x1, x2 = u, u_bar
y1, y2 = v * f_max, v_bar * f_max

# Contar aciertos en cada par
hits1 = (y1 <= f(x1)).astype(float)
hits2 = (y2 <= f(x2)).astype(float)

# Promediar aciertos en cada par (antitéticas)
hits_pairs = 0.5 * (hits1 + hits2)

I_hitmiss_anti = np.mean(hits_pairs) * f_max # Estimación de la integral
Var_hitmiss_anti = np.var(hits_pairs * f_max, ddof=1) / M # Varianza del estimador

t4_end = time.time()
t4 = t4_end - t4_start

print("=== Método Acierto y Error con Variables Antitéticas ===")
print(f"Estimación Integral = {I_hitmiss_anti:.6f}")
print(f"Varianza = {Var_hitmiss_anti:.6e}")
print()

#=== Comparación ===
reduction_pct = 100 * (Var_hitmiss - Var_hitmiss_anti) / Var_hitmiss
print("=== Comparación ===")
print(f"Reducción relativa de varianza = {reduction_pct:.2f} %")

=== Método Acierto y Error ===
Estimación Integral = 0.630258
Varianza = 1.612335e-06

=== Método Acierto y Error con Variables Antitéticas ===
Estimación Integral = 0.629230
Varianza = 9.566467e-07

=== Comparación ===
Reducción relativa de varianza = 40.67 %


# CONCLUSIONES
- MC Crudo resultó más eficiente y estable que Acierto y Error para el mismo tamaño de muestra.  
- La técnica de variables antitéticas redujo significativamente la varianza: ~86.8% en MC Crudo y ~40.7% en Acierto y Error.  
- La técnica de variables antitéticas es especialmente eficaz para funciones continuas y monotónicas, ya que permite disminuir notablemente la varianza con la misma cantidad de muestras.
- En estimadores discretos como Acierto y Error, la reducción es moderada y depende de la correlación entre los indicadores. Se recomienda evaluar empíricamente la reducción de varianza antes de asumir mejoras significativas.
